In [1]:
# -*- coding: utf-8 -*-
#
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
import json
import os
import pandas as pd
import torch
import numpy as np
from dgllife.data import UnlabeledSMILES
from dgllife.utils import mol_to_bigraph
from functools import partial
from torch.utils.data import DataLoader
from tqdm import tqdm
from utils import mkdir_p, collate_molgraphs_unlabeled, load_model, predict, init_featurizer
from sklearn.metrics import roc_curve,roc_auc_score, confusion_matrix, precision_recall_curve, auc, mean_squared_error, \
    r2_score, mean_absolute_error,cohen_kappa_score,accuracy_score,f1_score,matthews_corrcoef,precision_score,recall_score

# the metrics for classification
class AllMeter(object):
    
    def statistical(self):
        
        y_test = pd.DataFrame(self.y_test)
        y_predict = pd.DataFrame(self.y_predict)
        fpr, tpr, threshold = roc_curve(y_test, y_predict)
        auc_prc = auc(precision_recall_curve(y_test, y_predict, pos_label=1)[1],
                              precision_recall_curve(y_test, y_predict, pos_label=1)[0])
        auc_roc = auc(fpr, tpr)
        output_tran = []
        for x in y_predict[0]:
            if x > 0.5:
                output_tran.append(1)
            else:
                output_tran.append(0)
        acc = accuracy_score(y_test, output_tran)
        recall = recall_score(y_test, output_tran)
        precision = precision_score(y_test, output_tran)
        f1 = f1_score(y_test, output_tran)
        kappa = cohen_kappa_score(y_test,output_tran)   
        mcc = matthews_corrcoef(y_test,output_tran)
        
        c_mat = confusion_matrix(y_test, output_tran)            
        tn, fp, fn, tp = list(c_mat.flatten())
        se = tp / (tp + fn)
        sp = tn / (tn + fp)
        acc_ = (tp + tn) / (tn + fp + fn + tp)          
        recall_ = se
        print({'tn':tn, 'fp':fp, 'fn':fn, 'tp':tp})
        print(tp / (tp + fp))
        precision_ = tp / (tp + fp)
        f1_ = 2 * (precision * recall) / (precision + recall) # F1 = 2 * (precision * recall) / (precision + recall)
        mcc_ = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn) + 1e-8)
        scores_dict = {}
        scores_dict['acc'] = acc
        scores_dict['auc_roc'] = auc_roc
        scores_dict['recall'] = recall
        scores_dict['precision'] = precision
        scores_dict['f1'] = f1
        scores_dict['kappa'] = kappa
        scores_dict['mcc'] = mcc 
        scores_dict['auc_prc'] = auc_prc
        import collections
        scores_dict = collections.OrderedDict(scores_dict)
        print(scores_dict) 
        print({'混淆矩阵 acc':acc_,'recall_':recall_,'precision_':precision_,'f1_':f1_,'mcc_':mcc_})
        return scores_dict
         
    
    def __init__(self, mean=None, std=None):
        self.y_predict = []
        self.y_test = []
            
    def update(self, output, label, mask=None):
            output = torch.sigmoid(output)
            output = output.cpu().detach().numpy()
            label = np.array(label)
            for i in output:
                self.y_predict.append(i)
            for j in label:
                self.y_test.append(j)        
    def compute_metric(self, metric_name, reduction='mean'):
        if metric_name == 'getAllMetrics':
            return self.statistical()

def main(args):
    all_eval_meter = AllMeter()
    dataset = UnlabeledSMILES(args['smiles'], node_featurizer=args['node_featurizer'],
                              edge_featurizer=args['edge_featurizer'],
                              mol_to_graph=partial(mol_to_bigraph, add_self_loop=True))
#   batch size must be greater than or equal to predict data size
    dataloader = DataLoader(dataset, batch_size=args['batch_size'],
                            collate_fn=collate_molgraphs_unlabeled, num_workers=args['num_workers'])
    model = load_model(args).to(args['device'])
    checkpoint = torch.load(args['train_result_path'], map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    smiles_list = []
    predictions = []
    y_predict = []
    with torch.no_grad():
        for batch_id, batch_data in enumerate(tqdm(dataloader, desc="Iteration")):
            batch_smiles, bg = batch_data
            smiles_list.extend(batch_smiles)
            batch_pred = predict(args, model, bg)
            labels = pd.read_csv(args['file_path'])[args['task_names']].tolist()
            all_eval_meter.update(batch_pred, labels, mask=None)
            output = torch.sigmoid(batch_pred)
            output = output.cpu().detach().numpy()
            for i in output:
                predictions.append(i)     
    y_predict = pd.DataFrame(predictions)
    output_tran = []
    for x in y_predict[0]:
        if x > 0.5:
            output_tran.append(1)
        else:
            output_tran.append(0)
    all_scores = all_eval_meter.compute_metric('getAllMetrics')
    return y_predict,output_tran,all_scores

/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def loadConfiguration(args_interface):
# Load configuration
    with open(args_interface['train_result_path'] + '/configure.json', 'r') as f:
        args_interface.update(json.load(f))

    if torch.cuda.is_available():
        args_interface['device'] = torch.device('cuda:0')
    else:
        args_interface['device'] = torch.device('cpu')

    if args_interface['file_path'].endswith('.csv') or args_interface['file_path'].endswith('.csv.gz'):
        import pandas
        df = pandas.read_csv(args_interface['file_path'])
        if args_interface['smiles_column'] is not None:
            smiles = df[args_interface['smiles_column']].tolist()
        else:
            assert len(df.columns) == 1, 'The CSV file has more than 1 columns and ' \
                                         '-sc (smiles-column) needs to be specified.'
            smiles = df[df.columns[0]].tolist()
    elif args_interface['file_path'].endswith('.txt'):
        from dgllife.utils import load_smiles_from_txt
        smiles = load_smiles_from_txt(args_interface['file_path'])
    else:
        raise ValueError('Expect the input data file to be a .csv or a .txt file, '
                         'got {}'.format(args_interface['file_path']))
    args_interface['smiles'] = smiles
    args_interface = init_featurizer(args_interface)

    # Handle directories
    mkdir_p(args_interface['inference_result_path'])
    assert os.path.exists(args_interface['train_result_path']), \
        'The path to the saved training results does not exist.'
    return args_interface

In [3]:
from sklearn.metrics import roc_curve,roc_auc_score, confusion_matrix, precision_recall_curve, auc, mean_squared_error, \
    r2_score, mean_absolute_error,cohen_kappa_score,accuracy_score,f1_score,matthews_corrcoef,precision_score,recall_score

def getMetric(label, pre):
    return accuracy_score(label, pre)

In [4]:
def getResults(args_interface,modelName,repetitions,path):
    predict_data = pd.DataFrame()
    pre_result = []
    pre_scores = []
    for i in range(repetitions):
        args_interface['train_result_path'] = path
        args_interface['train_result_path'] = path+'/saved_model/{}{}.pth'.format(modelName,i+1)
        y_predictPro,y_predict,all_scores = main(args_interface)
        y_test = pd.read_csv(args_interface['file_path'])[args_interface['task_names']].tolist()
        acc = getMetric(y_test, y_predict)
        pre_result.append(acc)
        pre_scores.append(all_scores)
        predict_data['smiles'] = args_interface['smiles']
        predict_data['Test'] = y_test
        predict_data['label_model_'+str(i+1)] = y_predictPro

    print('acc:{}±{}'.format(np.mean(pre_result),np.std(pre_result)))
    with open(args_interface['inference_result_path'] + '/output_GAT.txt', 'w') as f:
        f.write('acc:{}±{}'.format(np.mean(pre_result),np.std(pre_result)))
    print(args_interface['inference_result_path'])

    data_df = pd.DataFrame(pre_scores)
    data_df.loc['Mean'] = data_df.mean()
    data_df.loc['Std'] = data_df.std()
    data_df.to_csv(args_interface['inference_result_path']+'predict_result.csv',index = False)
    predict_data['label'] = predict_data.iloc[:,2:].mean(axis=1)
    predict_data.to_csv(args_interface['inference_result_path']+'predict_data.csv',index = False)
    return data_df,predict_data

In [5]:
def statistical(y_true, y_pred, y_pro):
    c_mat = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = list(c_mat.flatten())
    se = tp / (tp + fn)
    sp = tn / (tn + fp)
    auc_prc = auc(precision_recall_curve(y_true, y_pro, pos_label=1)[1],
                  precision_recall_curve(y_true, y_pro, pos_label=1)[0])
    acc = (tp + tn) / (tn + fp + fn + tp)
#     acc_skl = accuracy_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pro)
    recall = se
#     recall_skl = recall_score(y_true, y_pred)
    precision = tp / (tp + fp)
#     precision_skl = precision_score(y_true, y_pred)
    f1 = 2 * (precision * recall) / (precision + recall) # F1 = 2 * (precision * recall) / (precision + recall)
#     f1_skl = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn) + 1e-8)
#     mcc_skl = matthews_corrcoef(y_true,y_pred)

    acc = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)   
    mcc = matthews_corrcoef(y_true, y_pred)

    scores_dict = {}
    scores_dict['acc'] = acc
    scores_dict['auc_roc'] = auc_roc
    scores_dict['recall'] = recall
    scores_dict['precision'] = precision
    scores_dict['f1'] = f1
    scores_dict['kappa'] = kappa
    scores_dict['mcc'] = mcc 
    scores_dict['auc_prc'] = auc_prc
    import collections
    scores_dict = collections.OrderedDict(scores_dict)
    
    return scores_dict

def GetPreditcTable(preDate,tureData,name,result_path):
    smiles = tureData['smiles'].to_list()
    y_true = tureData['label'].to_list()
    repetitions = 10
    predict_data = pd.DataFrame()
    predict_data['smiles'] = smiles
    predict_data['Test'] = y_true
    pre_scores = []
    y_pro_avg = preDate['label']
    y_pred_avg = [1 if p > 0.5 else 0 for p in y_pro_avg]  
    avg_scores_dict = statistical(y_true, y_pred_avg, y_pro_avg)
    avg_scores_dict = pd.DataFrame(avg_scores_dict,index=['avg'])
    for i in range(repetitions):
        y_pro = preDate['label_model_'+str(i+1)]
        y_pred = [1 if p > 0.5 else 0 for p in y_pro]
        predict_data['label_model_'+str(i+1)] = y_pred
        scores_dict = statistical(y_true, y_pred, y_pro)
        pre_scores.append(scores_dict)
   
    predict_data['label'] =  y_pred_avg
    predict_data.to_csv(result_path+name+'_Cal_Metric_Data_bak.csv',index = False)
    
    data_df = pd.DataFrame(pre_scores)
    data_df.loc['Mean'] = data_df.mean()
    data_df.loc['Std'] = data_df.std()
    data_df = pd.concat([data_df, avg_scores_dict], axis=0)
    data_df.to_csv(result_path+name+'_Cal_Metric_Summarize.csv',index = False)
    return data_df

In [6]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('-f', '--file-path', type=str, required=True,
                    help='Path to a .csv/.txt file of SMILES strings')
parser.add_argument('-sc', '--smiles-column', type=str,
                    help='Header for the SMILES column in the CSV file, can be '
                         'omitted if the input file is a .txt file or the .csv '
                         'file only has one column of SMILES strings')
parser.add_argument('-tp', '--train-result-path', type=str, default='classification_results',
                    help='Path to the saved training results, which will be used for '
                         'loading the trained model and related configurations')
parser.add_argument('-ip', '--inference-result-path', type=str, default='classification_inference_results',
                    help='Path to save the inference results')
parser.add_argument('-t', '--task-names',type=str,
                    help='Task names for saving model predictions in the CSV file to output, '
                         'which should be the same as the ones used for training. If not '
                         'specified, we will simply use task1, task2, ...')
parser.add_argument('-s', '--soft-classification', action='store_true', default=False,
                    help='By default we will perform hard classification with binary labels. '
                         'This flag allows performing soft classification instead.')
parser.add_argument('-nw', '--num-workers', type=int, default=1,
                    help='Number of processes for data loading (default: 1)')

_StoreAction(option_strings=['-nw', '--num-workers'], dest='num_workers', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, help='Number of processes for data loading (default: 1)', metavar=None)

In [7]:
args_interface = parser.parse_args(args=['--file-path','predict_data/6_pre_32_AR_剔除NURA重复数据.csv',
                               '--smiles-column','smiles',
                               '--task-names','label',
                               '--train-result-path','result/6108_AR_NURA_MPNN_0920',
                               '--inference-result-path','predict_result/6108_AR_pre_32_AR_10_MPNN_1008/'                              
                                  ]).__dict__

tureData = pd.read_csv(args_interface['file_path'])

In [8]:
args_interface['train_result_path'] = 'result/6108_AR_NURA_MPNN_0920'
args_interface['inference_result_path'] = 'predict_result/6108_AR_pre_32_AR_10_MPNN_1008/'
path = args_interface['train_result_path']
modelName = 'MPNN_bst_'
repetitions = 10
args_interface = loadConfiguration(args_interface)
data_df,predict_data = getResults(args_interface,modelName,repetitions,path)
result = GetPreditcTable(predict_data,tureData,modelName,args_interface['inference_result_path'])

Directory predict_result/6108_AR_pre_32_AR_10_MPNN_1008/ already exists.


Iteration: 100%|██████████| 1/1 [00:00<00:00,  2.80it/s]


{'tn': 11, 'fp': 6, 'fn': 9, 'tp': 6}
0.5
OrderedDict([('acc', 0.53125), ('auc_roc', 0.5764705882352941), ('recall', 0.4), ('precision', 0.5), ('f1', 0.4444444444444445), ('kappa', 0.04761904761904767), ('mcc', 0.048507125007266595), ('auc_prc', 0.5787615242261382)])
{'混淆矩阵 acc': 0.53125, 'recall_': 0.4, 'precision_': 0.5, 'f1_': 0.4444444444444445, 'mcc_': 0.04850712500726263}


Iteration: 100%|██████████| 1/1 [00:00<00:00, 18.23it/s]


{'tn': 16, 'fp': 1, 'fn': 12, 'tp': 3}
0.75
OrderedDict([('acc', 0.59375), ('auc_roc', 0.5882352941176471), ('recall', 0.2), ('precision', 0.75), ('f1', 0.31578947368421056), ('kappa', 0.14754098360655743), ('mcc', 0.2130214807490179), ('auc_prc', 0.6504846982320749)])
{'混淆矩阵 acc': 0.59375, 'recall_': 0.2, 'precision_': 0.75, 'f1_': 0.31578947368421056, 'mcc_': 0.21302148074898059}


...

Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.98it/s]


{'tn': 13, 'fp': 4, 'fn': 13, 'tp': 2}
0.3333333333333333
OrderedDict([('acc', 0.46875), ('auc_roc', 0.5372549019607843), ('recall', 0.13333333333333333), ('precision', 0.3333333333333333), ('f1', 0.19047619047619044), ('kappa', -0.10569105691056913), ('mcc', -0.1303589814791738), ('auc_prc', 0.5105642729887306)])
{'混淆矩阵 acc': 0.46875, 'recall_': 0.13333333333333333, 'precision_': 0.3333333333333333, 'f1_': 0.19047619047619044, 'mcc_': -0.13035898147915745}


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.31it/s]
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in long_scalars
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in long_scalars
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encou

{'tn': 17, 'fp': 0, 'fn': 15, 'tp': 0}
nan
OrderedDict([('acc', 0.53125), ('auc_roc', 0.5215686274509804), ('recall', 0.0), ('precision', 0.0), ('f1', 0.0), ('kappa', 0.0), ('mcc', 0.0), ('auc_prc', 0.4386788236889799)])
{'混淆矩阵 acc': 0.53125, 'recall_': 0.0, 'precision_': nan, 'f1_': nan, 'mcc_': 0.0}
acc:0.525±0.08244316223920575
predict_result/6108_AR_pre_32_AR_10_MPNN_1008/


/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in long_scalars
  del sys.path[0]
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [9]:
print(modelName)
result

MPNN_bst_


,acc,auc_roc,recall,precision,f1,kappa,mcc,auc_prc
0,0.531250,0.576471,0.400000,0.500000,0.444444,0.047619,0.048507,0.578762
1,0.593750,0.588235,0.200000,0.750000,0.315789,0.147541,0.213021,0.650485
2,0.531250,0.529412,0.733333,0.500000,0.594595,0.083969,0.092884,0.465396
3,0.406250,0.321569,0.133333,0.250000,0.173913,-0.225806,-0.253086,0.361838
4,0.718750,0.662745,0.666667,0.714286,0.689655,0.433071,0.433933,0.635279
5,0.531250,0.556863,0.200000,0.500000,0.285714,0.024390,0.030083,0.616316
6,0.437500,0.345098,0.266667,0.363636,0.307692,-0.147410,-0.152449,0.429325
7,0.500000,0.400000,0.000000,0.000000,0.000000,-0.062241,-0.168710,0.384697
8,0.468750,0.537255,0.133333,0.333333,0.190476,-0.105691,-0.130359,0.510564
9,0.531250,0.521569,0.000000,0.000000,0.000000,0.000000,0.000000,0.438679


In [10]:
args_interface['train_result_path'] = 'result/6108_AR_NURA_AFP_0920'
args_interface['inference_result_path'] = 'predict_result/6108_AR_pre_32_AR_10_AFP_1008/'
path = args_interface['train_result_path']
modelName = 'AttentiveFP_bst_'
repetitions = 10
args_interface = loadConfiguration(args_interface)
data_df,predict_data = getResults(args_interface,modelName,repetitions,path)
result = GetPreditcTable(predict_data,tureData,modelName,args_interface['inference_result_path'])

Directory predict_result/6108_AR_pre_32_AR_10_AFP_1008/ already exists.


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.49it/s]


{'tn': 13, 'fp': 4, 'fn': 3, 'tp': 12}
0.75
OrderedDict([('acc', 0.78125), ('auc_roc', 0.8666666666666667), ('recall', 0.8), ('precision', 0.75), ('f1', 0.7741935483870969), ('kappa', 0.5625), ('mcc', 0.5636018619766345), ('auc_prc', 0.7492884990253412)])
{'混淆矩阵 acc': 0.78125, 'recall_': 0.8, 'precision_': 0.75, 'f1_': 0.7741935483870969, 'mcc_': 0.5636018619765913}


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.83it/s]

{'tn': 12, 'fp': 5, 'fn': 6, 'tp': 9}
0.6428571428571429
OrderedDict([('acc', 0.65625), ('auc_roc', 0.7372549019607844), ('recall', 0.6), ('precision', 0.6428571428571429), ('f1', 0.6206896551724138), ('kappa', 0.30708661417322836), ('mcc', 0.3076976944152481), ('auc_prc', 0.7543937061328365)])
{'混淆矩阵 acc': 0.65625, 'recall_': 0.6, 'precision_': 0.6428571428571429, 'f1_': 0.6206896551724138, 'mcc_': 0.30769769441522415}


...

{'tn': 10, 'fp': 7, 'fn': 4, 'tp': 11}
0.6111111111111112
OrderedDict([('acc', 0.65625), ('auc_roc', 0.788235294117647), ('recall', 0.7333333333333333), ('precision', 0.6111111111111112), ('f1', 0.6666666666666666), ('kappa', 0.3178294573643411), ('mcc', 0.3234770633596198), ('auc_prc', 0.7984612636071446)])
{'混淆矩阵 acc': 0.65625, 'recall_': 0.7333333333333333, 'precision_': 0.6111111111111112, 'f1_': 0.6666666666666666, 'mcc_': 0.3234770633595946}


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.72it/s]


{'tn': 17, 'fp': 0, 'fn': 4, 'tp': 11}
1.0
OrderedDict([('acc', 0.875), ('auc_roc', 0.8627450980392157), ('recall', 0.7333333333333333), ('precision', 1.0), ('f1', 0.846153846153846), ('kappa', 0.7450199203187251), ('mcc', 0.7704873741021288), ('auc_prc', 0.8966719672274367)])
{'混淆矩阵 acc': 0.875, 'recall_': 0.7333333333333333, 'precision_': 1.0, 'f1_': 0.846153846153846, 'mcc_': 0.7704873741020634}


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.58it/s]


{'tn': 13, 'fp': 4, 'fn': 5, 'tp': 10}
0.7142857142857143
OrderedDict([('acc', 0.71875), ('auc_roc', 0.7450980392156863), ('recall', 0.6666666666666666), ('precision', 0.7142857142857143), ('f1', 0.689655172413793), ('kappa', 0.4330708661417323), ('mcc', 0.43393264597022163), ('auc_prc', 0.744919825333992)])
{'混淆矩阵 acc': 0.71875, 'recall_': 0.6666666666666666, 'precision_': 0.7142857142857143, 'f1_': 0.689655172413793, 'mcc_': 0.4339326459701879}
acc:0.690625±0.0820275296775418
predict_result/6108_AR_pre_32_AR_10_AFP_1008/


In [11]:
print(modelName)
result

AttentiveFP_bst_


,acc,auc_roc,recall,precision,f1,kappa,mcc,auc_prc
0,0.781250,0.866667,0.800000,0.750000,0.774194,0.562500,0.563602,0.749288
1,0.656250,0.737255,0.600000,0.642857,0.620690,0.307087,0.307698,0.754394
2,0.687500,0.760784,0.733333,0.647059,0.687500,0.377432,0.380392,0.694250
3,0.687500,0.866667,0.800000,0.631579,0.705882,0.382239,0.394472,0.885196
4,0.562500,0.713725,0.333333,0.555556,0.416667,0.100402,0.108814,0.601178
5,0.656250,0.725490,0.666667,0.625000,0.645161,0.312500,0.313112,0.733440
6,0.625000,0.678431,0.533333,0.615385,0.571429,0.241107,0.243059,0.718058
7,0.656250,0.788235,0.733333,0.611111,0.666667,0.317829,0.323477,0.798461
8,0.875000,0.862745,0.733333,1.000000,0.846154,0.745020,0.770487,0.896672
9,0.718750,0.745098,0.666667,0.714286,0.689655,0.433071,0.433933,0.744920


In [12]:
args_interface['train_result_path'] = 'result/6108_AR_NURA_GAT_0920'
args_interface['inference_result_path'] = 'predict_result/6108_AR_pre_32_AR_10_GAT_1008/'
path = args_interface['train_result_path']
modelName = 'GAT_bst_'
repetitions = 10
args_interface = loadConfiguration(args_interface)
data_df,predict_data = getResults(args_interface,modelName,repetitions,path)
result = GetPreditcTable(predict_data,tureData,modelName,args_interface['inference_result_path'])

Directory predict_result/6108_AR_pre_32_AR_10_GAT_1008/ already exists.


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.42it/s]
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in long_scalars
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in long_scalars
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encou

{'tn': 17, 'fp': 0, 'fn': 15, 'tp': 0}
nan
OrderedDict([('acc', 0.53125), ('auc_roc', 0.6705882352941176), ('recall', 0.0), ('precision', 0.0), ('f1', 0.0), ('kappa', 0.0), ('mcc', 0.0), ('auc_prc', 0.6961146494164432)])
{'混淆矩阵 acc': 0.53125, 'recall_': 0.0, 'precision_': nan, 'f1_': nan, 'mcc_': 0.0}


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.17it/s]
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in long_scalars
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in long_scalars
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encou

{'tn': 17, 'fp': 0, 'fn': 15, 'tp': 0}
nan
OrderedDict([('acc', 0.53125), ('auc_roc', 0.6823529411764706), ('recall', 0.0), ('precision', 0.0), ('f1', 0.0), ('kappa', 0.0), ('mcc', 0.0), ('auc_prc', 0.6980106274282716)])
{'混淆矩阵 acc': 0.53125, 'recall_': 0.0, 'precision_': nan, 'f1_': nan, 'mcc_': 0.0}


...

{'tn': 17, 'fp': 0, 'fn': 15, 'tp': 0}
nan
OrderedDict([('acc', 0.53125), ('auc_roc', 0.6549019607843136), ('recall', 0.0), ('precision', 0.0), ('f1', 0.0), ('kappa', 0.0), ('mcc', 0.0), ('auc_prc', 0.6520588026827395)])
{'混淆矩阵 acc': 0.53125, 'recall_': 0.0, 'precision_': nan, 'f1_': nan, 'mcc_': 0.0}


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.39it/s]
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in long_scalars
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in long_scalars
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encou

{'tn': 17, 'fp': 0, 'fn': 15, 'tp': 0}
nan
OrderedDict([('acc', 0.53125), ('auc_roc', 0.7803921568627451), ('recall', 0.0), ('precision', 0.0), ('f1', 0.0), ('kappa', 0.0), ('mcc', 0.0), ('auc_prc', 0.7930208842927483)])
{'混淆矩阵 acc': 0.53125, 'recall_': 0.0, 'precision_': nan, 'f1_': nan, 'mcc_': 0.0}


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.59it/s]
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in long_scalars
  del sys.path[0]
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in long_scalars
  del sys.path[0]
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classifi

{'tn': 16, 'fp': 1, 'fn': 8, 'tp': 7}
0.875
OrderedDict([('acc', 0.71875), ('auc_roc', 0.796078431372549), ('recall', 0.4666666666666667), ('precision', 0.875), ('f1', 0.608695652173913), ('kappa', 0.4193548387096774), ('mcc', 0.4700159919404217), ('auc_prc', 0.814612790295743)])
{'混淆矩阵 acc': 0.71875, 'recall_': 0.4666666666666667, 'precision_': 0.875, 'f1_': 0.608695652173913, 'mcc_': 0.47001599194037375}
acc:0.5625±0.05929270612815711
predict_result/6108_AR_pre_32_AR_10_GAT_1008/


In [13]:
print(modelName)
result

GAT_bst_


,acc,auc_roc,recall,precision,f1,kappa,mcc,auc_prc
0,0.531250,0.670588,0.000000,0.000000,0.000000,0.000000,0.000000,0.696115
1,0.531250,0.682353,0.000000,0.000000,0.000000,0.000000,0.000000,0.698011
2,0.625000,0.811765,0.266667,0.800000,0.400000,0.216327,0.285653,0.768949
3,0.531250,0.709804,0.000000,0.000000,0.000000,0.000000,0.000000,0.716641
4,0.562500,0.792157,0.066667,1.000000,0.125000,0.070539,0.191204,0.761347
5,0.531250,0.619608,0.000000,0.000000,0.000000,0.000000,0.000000,0.696006
6,0.531250,0.768627,0.000000,0.000000,0.000000,0.000000,0.000000,0.785086
7,0.531250,0.654902,0.000000,0.000000,0.000000,0.000000,0.000000,0.652059
8,0.531250,0.780392,0.000000,0.000000,0.000000,0.000000,0.000000,0.793021
9,0.718750,0.796078,0.466667,0.875000,0.608696,0.419355,0.470016,0.814613


In [14]:
args_interface['train_result_path'] = 'result/6108_AR_NURA_GCN_0920'
args_interface['inference_result_path'] = 'predict_result/6108_AR_pre_32_AR_10_GCN_1008/'
path = args_interface['train_result_path']
modelName = 'GCN_bst_'
repetitions = 10
args_interface = loadConfiguration(args_interface)
data_df,predict_data = getResults(args_interface,modelName,repetitions,path)
result = GetPreditcTable(predict_data,tureData,modelName,args_interface['inference_result_path'])

Directory predict_result/6108_AR_pre_32_AR_10_GCN_1008/ already exists.


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.45it/s]


{'tn': 5, 'fp': 12, 'fn': 2, 'tp': 13}
0.52
OrderedDict([('acc', 0.5625), ('auc_roc', 0.6549019607843137), ('recall', 0.8666666666666667), ('precision', 0.52), ('f1', 0.65), ('kappa', 0.15471698113207544), ('mcc', 0.19408623801577188), ('auc_prc', 0.545489962750574)])
{'混淆矩阵 acc': 0.5625, 'recall_': 0.8666666666666667, 'precision_': 0.52, 'f1_': 0.65, 'mcc_': 0.19408623801575012}


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.81it/s]


{'tn': 13, 'fp': 4, 'fn': 4, 'tp': 11}
0.7333333333333333
OrderedDict([('acc', 0.75), ('auc_roc', 0.8274509803921568), ('recall', 0.7333333333333333), ('precision', 0.7333333333333333), ('f1', 0.7333333333333333), ('kappa', 0.4980392156862745), ('mcc', 0.4980392156862745), ('auc_prc', 0.7025315730494759)])
{'混淆矩阵 acc': 0.75, 'recall_': 0.7333333333333333, 'precision_': 0.7333333333333333, 'f1_': 0.7333333333333333, 'mcc_': 0.4980392156862362}


...

{'tn': 9, 'fp': 8, 'fn': 2, 'tp': 13}
0.6190476190476191
OrderedDict([('acc', 0.6875), ('auc_roc', 0.8666666666666666), ('recall', 0.8666666666666667), ('precision', 0.6190476190476191), ('f1', 0.7222222222222222), ('kappa', 0.3869731800766284), ('mcc', 0.4161455870818984), ('auc_prc', 0.7565450845450846)])
{'混淆矩阵 acc': 0.6875, 'recall_': 0.8666666666666667, 'precision_': 0.6190476190476191, 'f1_': 0.7222222222222222, 'mcc_': 0.41614558708186306}


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.63it/s]


{'tn': 6, 'fp': 11, 'fn': 4, 'tp': 11}
0.5
OrderedDict([('acc', 0.53125), ('auc_roc', 0.5490196078431372), ('recall', 0.7333333333333333), ('precision', 0.5), ('f1', 0.5945945945945945), ('kappa', 0.08396946564885499), ('mcc', 0.09288407280256479), ('auc_prc', 0.5229203647323276)])
{'混淆矩阵 acc': 0.53125, 'recall_': 0.7333333333333333, 'precision_': 0.5, 'f1_': 0.5945945945945945, 'mcc_': 0.09288407280255652}


Iteration: 100%|██████████| 1/1 [00:00<00:00, 21.70it/s]
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/admin/anaconda3/envs/dgllife/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


{'tn': 0, 'fp': 17, 'fn': 0, 'tp': 15}
0.46875
OrderedDict([('acc', 0.46875), ('auc_roc', 0.5843137254901961), ('recall', 1.0), ('precision', 0.46875), ('f1', 0.6382978723404256), ('kappa', 0.0), ('mcc', 0.0), ('auc_prc', 0.5000371251279916)])
{'混淆矩阵 acc': 0.46875, 'recall_': 1.0, 'precision_': 0.46875, 'f1_': 0.6382978723404256, 'mcc_': 0.0}
acc:0.60625±0.1
predict_result/6108_AR_pre_32_AR_10_GCN_1008/


In [15]:
print(modelName)
result

GCN_bst_


,acc,auc_roc,recall,precision,f1,kappa,mcc,auc_prc
0,0.56250,0.654902,0.866667,0.520000,0.650000,0.154717,0.194086,0.545490
1,0.75000,0.827451,0.733333,0.733333,0.733333,0.498039,0.498039,0.702532
2,0.71875,0.749020,0.933333,0.636364,0.756757,0.450382,0.498196,0.636695
3,0.59375,0.619608,0.466667,0.583333,0.518519,0.174603,0.177859,0.499755
4,0.65625,0.764706,0.733333,0.611111,0.666667,0.317829,0.323477,0.656212
5,0.43750,0.701961,0.933333,0.451613,0.608696,-0.062731,-0.191204,0.600193
6,0.65625,0.741176,0.800000,0.600000,0.685714,0.323077,0.339550,0.649301
7,0.68750,0.866667,0.866667,0.619048,0.722222,0.386973,0.416146,0.756545
8,0.53125,0.549020,0.733333,0.500000,0.594595,0.083969,0.092884,0.522920
9,0.46875,0.584314,1.000000,0.468750,0.638298,0.000000,0.000000,0.500037
